In [63]:
import warnings
warnings.filterwarnings('ignore')

In [64]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [65]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [66]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [ "loan_amnt", "int_rate", "installment","annual_inc", "loan_status", "dti", "delinq_2yrs", "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "total_acc", "out_prncp", "out_prncp_inv", "total_pymnt","total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee","recoveries","collection_recovery_fee", "last_pymnt_amnt", "collections_12_mths_ex_med", "policy_code", "acc_now_delinq","tot_coll_amt","tot_cur_bal", "open_acc_6m", "open_act_il","open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il","il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc","all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl","inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct","mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc","mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd","num_actv_bc_tl","num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl","num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0","num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m","num_tl_op_past_12m","pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies","tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit","total_il_high_credit_limit"]
   

target = ["loan_status"]

In [67]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,35000.0,0.1308,797.80,125890.0,Fully Paid,30.48,0.0,0.0,14.0,0.0,...,0.0,2.0,100.0,80.0,0.0,0.0,399376.0,131350.0,32700.0,130876.0
1,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0
2,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0
3,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0
4,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0


# Split the Data into Training and Testing

In [68]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [69]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,70423.000000,70423.000000,70423.000000,7.042300e+04,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,...,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.0,7.042300e+04,7.042300e+04,70423.000000,7.042300e+04
mean,16621.372989,0.128063,479.478688,8.843486e+04,21.745274,0.217244,0.500632,12.583787,0.125939,17549.650966,...,0.052213,2.231657,95.073013,30.515630,0.125882,0.0,2.100903e+05,6.131966e+04,29748.067038,5.572026e+04
std,10282.268052,0.048339,288.457221,1.218578e+05,20.429814,0.718822,0.760803,6.028804,0.336711,21880.270487,...,0.393946,1.907077,8.311660,33.618896,0.336648,0.0,1.928094e+05,5.735407e+04,26888.846929,5.090730e+04
min,1000.000000,0.060000,30.640000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,264.930000,5.000000e+04,13.840000,0.000000,0.000000,8.000000,0.000000,6240.500000,...,0.000000,1.000000,93.100000,0.000000,0.000000,0.0,6.700800e+04,2.645200e+04,11600.000000,2.286200e+04
50%,15000.000000,0.118000,402.890000,7.300000e+04,19.710000,0.000000,0.000000,11.000000,0.000000,12014.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.469600e+05,4.533800e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,646.740000,1.040000e+05,26.620000,0.000000,1.000000,16.000000,0.000000,21657.500000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036355e+05,7.656750e+04,39300.000000,7.250600e+04
max,40000.000000,0.308400,1676.230000,9.000000e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [70]:
# Check the balance of our target values
y.value_counts()

low_risk       68470
Fully Paid      1584
high_risk        347
Charged Off       22
Name: loan_status, dtype: int64

In [71]:
# Split the X and y into X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(52817, 76)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [72]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [73]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler = StandardScaler()

In [74]:
# Scale the training and testing data
X_Scaler = scaler.fit(X_train)
X_train_scaled = X_Scaler.transform(X_train)
X_test_scaled = X_Scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [75]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
BRFC = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
BRFC_model = BRFC.fit(X_train_scaled, y_train)
predictions = BRFC_model.predict(X_test_scaled)

In [76]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, predictions)

0.8310996902928265

In [77]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[    5,     0,     0,     0],
       [    8,   387,     0,     1],
       [    2,     1,    41,    43],
       [   86,   135,  1904, 14993]])

In [78]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.05      1.00      0.99      0.09      1.00      1.00         5
 Fully Paid       0.74      0.98      0.99      0.84      0.98      0.97       396
  high_risk       0.02      0.47      0.89      0.04      0.65      0.40        87
   low_risk       1.00      0.88      0.91      0.93      0.89      0.79     17118

avg / total       0.99      0.88      0.91      0.93      0.89      0.80     17606



In [79]:
# List the features sorted in descending order by feature importance
sorted(zip(BRFC_model.feature_importances_, X.columns), reverse=True)

[(0.09804120333226851, 'total_pymnt_inv'),
 (0.09325769836987158, 'total_pymnt'),
 (0.0856087891469024, 'total_rec_prncp'),
 (0.08321027751900076, 'out_prncp_inv'),
 (0.08081318020832641, 'out_prncp'),
 (0.06467960481142396, 'last_pymnt_amnt'),
 (0.060330458072162634, 'total_rec_int'),
 (0.018163217296167773, 'total_rev_hi_lim'),
 (0.014366184154209575, 'loan_amnt'),
 (0.013243512895949168, 'max_bal_bc'),
 (0.0124297874675847, 'bc_open_to_buy'),
 (0.012307475867010673, 'il_util'),
 (0.011835820486034685, 'num_sats'),
 (0.011530508634832173, 'mths_since_recent_bc'),
 (0.011237896975078421, 'installment'),
 (0.011090952303405174, 'num_il_tl'),
 (0.011082832780655652, 'int_rate'),
 (0.010939181957556507, 'bc_util'),
 (0.01078347030236533, 'mo_sin_rcnt_tl'),
 (0.01044538083837625, 'total_acc'),
 (0.0101041483537128, 'mo_sin_old_rev_tl_op'),
 (0.009935342999221426, 'open_acc'),
 (0.009751034238267791, 'avg_cur_bal'),
 (0.009588363314486113, 'annual_inc'),
 (0.009104550326935659, 'total_bc_l

### Easy Ensemble Classifier

In [80]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
EEC = EasyEnsembleClassifier(random_state=1)
EEC.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=1)

In [81]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.8310996902928265

In [82]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, predictions)

NameError: name 'model' is not defined

In [46]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, predictions))